In [858]:
import pandas as pd

df = pd.read_csv('trámites_crudos.csv')
df.head()

,descripción,organismos,título
0,Toda Persona Física o Jurídica que pretenda im...,#Ventanilla Única de Comercio Exterior - VUCE ...,Importación de vinos
1,Toda Persona Física o Jurídica que pretenda im...,#Ventanilla Única de Comercio Exterior - VUCE ...,Importación de alcohol
2,Para poder exportar productos vitivinícolas (d...,#Ventanilla Única de Comercio Exterior - VUCE ...,Inscripción como Exportador y/o importador de ...
3,Para poder exportar productos vitivinícolas (d...,#Instituto Nacional de Vitivinicultura - INV #...,Inscripción como Fabricante de Productos de Us...
4,Toda Persona Física o Jurídica que pretenda im...,#Instituto Nacional de Vitivinicultura - INV #...,Importación de pequeños volúmenes de vino


In [859]:
import re

def arreglar_texto(s):
    s = s.lower()
    
    # acrónimos
    s = re.sub(r'(?<=\W)(instituto nacional de vitivinicultura|inv|i.n.v.|i.n.v)(?=\W)', 'INV', s)
    s = re.sub(r'(?<=\W)(cuit|c.u.i.t.|c.u.i.t)(?=\W)', 'CUIT', s)
    s = re.sub(r'(?<=\W)(ig|i.g.|i.g)(?=\W)', 'IG', s)
    s = re.sub(r'(?<=\W)(doc|d.o.c.|d.o.c)(?=\W)', 'DOC', s)
    
    # repetitivo
    s = re.sub('para realizar el tr(á|a)mite (debe|deberá) adjuntar la siguiente documentaci(ó|o)n:', '', s)
    
    # falso interlineado
    s = re.sub(r'(?<!.)\n', ' ', s)
    
    # espacios y saltos de línea extras
    #s = re.sub(r'(?<=\w)\n+(?=\w)', ' ', s)
    s = re.sub(r'^\s+', '', s)
    s = re.sub(r' +', ' ', s)
    s = re.sub(r'\n *', '\n', s)
    s = re.sub(r'\n+', '\n', s)
    
    # puntuación
    s = re.sub(r' +(?=[,.:)])', '', s)
    s = re.sub(r'(?<=\() +', '', s)
    
    # párrafos
    s = re.sub(r'\n', '\n\n', s)
    
    # abreviaciones
    s = re.sub(r'(nº|n°) *', 'nº ', s)
    
    
    # mayúsculas
    callback = lambda match: match.group(0).upper()
    s = re.sub(r'(?<=\. )\w', callback, s)
    s = re.sub(r'(?<=\n)\w', callback, s)
    s = re.sub(r'^\w', callback, s)
    
    return s

In [860]:
df.título = df.título.apply(arreglar_texto)
df.descripción = df.descripción.apply(arreglar_texto)

In [862]:
df.drop(columns='organismos', inplace=True)

In [879]:
from pathlib import Path

carpeta = Path('trámites')
carpeta.mkdir(parents=True, exist_ok=True)

In [887]:
for _, trámite in df.iterrows():
    
    nombre_archivo = (
        '_'.join(
            trámite.título
            .lower()
            .replace('/', '_')
            .replace(',', '')
            .split(' ')
            [:7]
        )
        + '.md'
    )
    
    with open(carpeta / nombre_archivo, 'w') as f:
        f.write('# ' + trámite.título + '\n\n')
        f.write(trámite.descripción)